# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **  This is a classification problem.  This is because a pass/fail label is a discrete featureset, rather than a continuous one.  We are not modelling continuous data which would be more suited to regression.

A more advanced problem seeking to improve a student's *grades* might instead be a regression problem.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [37]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [38]:
# Calculate number of students
n_students = len(student_data.index)

# Calculate number of features, excluding the label column
n_features = len(student_data.columns) - 1

# Calculate passing students
n_passed = len(student_data[student_data['passed'] == 'yes'])

# Calculate failing students.  
# We do a full calculation in case a new value type is ever added e.g, 'pending', rather than n_students - n_passed
n_failed = len(student_data[student_data['passed'] == 'no'])

# Calculate graduation rate
grad_rate = n_passed / float(n_students) * 100.0
# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

print "\nF1 score for predicting all 'yes': {:.4f}".format(
    f1_score(y_true = ['yes']*n_passed + ['no']*n_failed, y_pred = ['yes']*n_students, pos_label='yes', average='binary'))

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%

F1 score for predicting all 'yes': 0.8030


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [39]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [40]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [41]:
# Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split

# Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all,y_all,train_size=num_train,test_size=num_test,random_state=0,stratify=y_all)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **

All models have been selected due to their inherent support for Classification type relationships.  Three models have been selected - Naive Bayes, Decision Trees and Support Vector Machines with some key information on each<sup>[1][2]</sup>:
<h3>Naive Bayes (GaussianNB)</h3>
- This model has been used for medical and biological applications, for example to predict DNA-binding proteins to help advance our  understanding of protein functions.<sup>[3]</sup>  Comparing other machine learning frameworks, the model performed best due to its abililty to handle lots of features.
- **Strengths:**  Naive Bayes performs well when inputs are independent from one another, and when there are many features
    - Few parameters so it's hard to overfit
    - Fast training speed
    - Good at seperating signal from noise
    - Able to handle many features
- **Weaknesses:** Naive Bayes performs badly for problems where probabilities are sometimes greater than zero for a particular classification.
    - Simplistic nature can yield bad generalizations or suboptimal solutions
    - Doesn't account for interactions between features
    - There is a high bias when there is a small amount of data
- **Applicability:** Our student data has a lot of features - 30 in total - which could influence the pass rate.  Naive Bayes could be a good candidate on this basis.  Assuming a typical school environment where high-end hardware is not available for generating the model, this will be a good candidate for lightweight learning.

<h3>Decision Trees</h3>
- This model has been used for Speech Recognition technologies, and recently applied to develop models for accented speech<sup>[4]</sup>.  By optimizing the tree and/or pruning it, algorithms have been improved for speech-recognition applications.  A key advantage of Decision Trees for accented speech was the favourable performance for adaptive data techniques - that is, using data from other models and implementing them into a current model.  
- **Strengths:** Decision trees perform well with certain boolean functions, and when the model can be built with smaller decision trees rather than bigger trees.
    - Fast training speed
    - Easy visualization
- **Weaknesses:** Decision trees perform badly if the tree grows quickly, and can overfit 
    - Trees can grow very fast
    - High memory footprint
    - Overfitting happens very easily
    - Generalize relatively poorly
- **Applicability:** Considering we have a large number of "yes"/"no" (i.e boolean) features in our dataset, this is well suited to a decision tree.  Moreover, we only have 395 data points, there is not a high risk of intense memory and CPU usage.  Even if the student base increases, it would likely stay in the same order of magnitude.  It is also easy to visualize whcih would be great for intepretation by the school board.

<h3>Support Vector Machines<sup>[5]</sup></h3>
- This model has been applied to the stock market to forecast movement of different markets, and found to work well in markets with either high volatility or lower volatility<sup>[6]</sup>.  In this case, Support Vector Machines were chosen because they tend to outperform other methods for financial time-series forecasting.
- **Strengths:** Support Vector Machines perform well when there is a definite distinction between two classes
    - Fast training speed
    - Good for feature-intense data, e.g. text
- **Weaknesses:** Support Vector Machines perform badly when the classification problem is not binary
    - Doesn't take into account all data equally, so can't see overall trends in data
    - A good choice in kernel is required, which is not always clear
    - Slow in test phase
- **Applicability:** Considering we have a binary classification - pass or fail - Support Vector Machines become a good candidate as a model.  Once again, assuming a typical school environment where high-end hardware is not available for generating the model, this will be a good candidate for relatively lightweight learning.

<hr>

[1]K. Markham, "Comparing supervised learning algorithms", Data School, 2015. [Online]. Available: http://www.dataschool.io/comparing-supervised-learning-algorithms/. [Accessed: 15- Nov- 2016].<br/>
[2]B. Rohrer, "How to choose machine learning algorithms", Docs.microsoft.com, 2016. [Online]. Available: https://docs.microsoft.com/en-us/azure/machine-learning/machine-learning-algorithm-choice. [Accessed: 15- Nov- 2016].<br/>
[3]W.  Lou, X.  Wang, F.  Chen, Y.  Chen, B.  Jiang and H.  Zhang, "Sequence Based Prediction of DNA-Binding Proteins Based on Hybrid Feature Selection Using Random Forest and Gaussian Naïve Bayes", PLoS ONE, vol. 9, no. 1, p. e86703, 2014.<br/>
[4]U.  Nallasamy, F.  Metze and T.  Schultz, "Enhanced Polyphone Decision Tree Adaptation for Accented Speech Recognition", Proceedings of INTERSPEECH, pp. 1902-1905, 2013.<br/>
[5]C. Burges, "A Tutorial on Support Vector Machines for Pattern Recognition", Data Mining and Knowledge Discovery, vol. 2, pp. 121-167, 2016.<br/>
[6]R. Rosillo, J. Giner, J. Puente and B. Ponte, "Different Stock Market Models Using Support Vector Machines", International Journal of Trade, Economics and Finance, pp. 310-313, 2013.<br/>

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [42]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    tdiff = end - start
    
    # Print and return the results
    print "Trained model in {:.4f} seconds".format(tdiff)
    return tdiff

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    tdiff = end - start
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(tdiff)
    return f1_score(target.values, y_pred, pos_label='yes'),tdiff


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_time = train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing, and return them
    f1_train, prediction_time_train = predict_labels(clf, X_train, y_train)
    print "F1 score for training set: {:.4f}.".format(f1_train)
    f1_test, prediction_time_test = predict_labels(clf, X_test, y_test)
    print "F1 score for test set: {:.4f}.".format(f1_test)
    return train_time,prediction_time_test,f1_train,f1_test

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [43]:
# Import the three supervised learning models from sklearn
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.utils import resample

# Get a benchmark score
print "F1 score for predicting all \"yes\" on test set: {:.4f}".format(
    f1_score(y_test, ['yes']*len(y_test), pos_label='yes', average='binary'))

# Initialize and setup the three models
classifiers = [
    { 'name' : "GaussianNB", 'clf' : GaussianNB() },
    { 'name' : "Decision Tree", 'clf' : DecisionTreeClassifier(criterion="entropy",random_state=0) },
    { 'name' : "Support Vector Machines", 'clf' : SVC(random_state=0) },
]

# Resample and store our training sets in specific sizes, in training_sets for 100,200 and 300
training_sets = []
for train_size in range(100,300+1,100):
    X_res, y_res = resample(X_train,y_train,n_samples=train_size,random_state=0)
    training_sets.append ({ 'size' : train_size, 'X_train' : X_res, 'y_train' : y_res })

# Loop through each classifer,  and each training set size and test our model
for clfData in classifiers:
    clfData['results'] = []
    for tset in training_sets:
        train_time,prediction_time_test,f1_train,f1_test = \
        train_predict(clfData['clf'], tset['X_train'], tset['y_train'], X_test, y_test)
        
        # Store our results
        clfData['results'].append({'train_time' : train_time, 'prediction_time_test' : prediction_time_test,
                                   'f1_train' : f1_train, 'f1_test' : f1_test, 'size' : tset['size']})
        
# Generate markdown
print '----- MARKDOWN -----'
i = 0
for clfData in classifiers:
    i += 1
    print '** Classifer {} - {}**\n'.format(i,clfData['name'])
    print '| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |'
    print '| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |'
    for result in clfData['results']:
        print '| {} | {:.4f} | {:.4f} | {:.4f} | {:.4f} |'.format(result['size'], result['train_time'], 
                                                       result['prediction_time_test'],result['f1_train'],result['f1_test'])
    print '\n'

F1 score for predicting all "yes" on test set: 0.8050
Training a GaussianNB using a training set size of 100. . .
Trained model in 0.0070 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 0.4583.
Made predictions in 0.0020 seconds.
F1 score for test set: 0.4270.
Training a GaussianNB using a training set size of 200. . .
Trained model in 0.0040 seconds
Made predictions in 0.0030 seconds.
F1 score for training set: 0.8681.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.6612.
Training a GaussianNB using a training set size of 300. . .
Trained model in 0.0060 seconds
Made predictions in 0.0070 seconds.
F1 score for training set: 0.8364.
Made predictions in 0.0030 seconds.
F1 score for test set: 0.6891.
Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0080 seconds
Made predictions in 0.0020 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.7218.
Training a Dec

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - GaussianNB**

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100 | 0.0070 | 0.0020 | 0.4583 | 0.4270 |
| 200 | 0.0040 | 0.0010 | 0.8681 | 0.6612 |
| 300 | 0.0060 | 0.0030 | 0.8364 | 0.6891 |


** Classifer 2 - Decision Tree**

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100 | 0.0080 | 0.0010 | 1.0000 | 0.7218 |
| 200 | 0.0110 | 0.0010 | 1.0000 | 0.6500 |
| 300 | 0.0080 | 0.0010 | 1.0000 | 0.6875 |


** Classifer 3 - Support Vector Machines**

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100 | 0.0060 | 0.0030 | 0.9193 | 0.8105 |
| 200 | 0.0340 | 0.0070 | 0.9241 | 0.8052 |
| 300 | 0.0260 | 0.0090 | 0.9214 | 0.8366 |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

Generally speaking, all training times are in the same order of magnitude, with Support Vector Machines being the highest.  In terms of Prediction times, Support Vector Machines was also generally the highest.

Despite this, the maximal time is still less than a millisecond and is unlikely to cause problems even for schools with modest-hardware.  Hence, it is reccomended to choose the Support Vector Machines as a model. It has the most consistent performance regardless of training set size, and consistently delivers high F<sub>1</sub> scores around 0.78.  On the other hand, Decision Tree delivers lower, less robust performance from 0.65-0.72 and GaussianNB delivers a very un-robust and poor score of 0.42-0.75.  Clearly, Support Vector Machines have the most robust and strongest model going forward.

Future training set sizes are unlikely to increase by an order of magnitude so the prediction and training times are unlikely to increase drastically.  Moreover, even with a large training and prediction time, it is likely that the model would be trained and used on occasion at the end of each academic year rather than persistently.  This is because student performance and new school strategies are likely to be evaluated thoroughly once per year.  Considering this, performance and time are unlikely to be mission-critical even if order of magnitudes higher (i.e. seconds).  This means it is fine to stick with Support Vector Machines despite any performance concerns.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **

Support Vector Machines (SVM) basically work by trying to identify the reasons why a student switches from a 'fail' to a 'pass'.  In other words, it isn't very concerned with people who are comfortably passing or failing, but rather what the differences are between students who have only *just* passed, or only *just* failed.   It will try to establish this *boundary* between marginal students as clearly as possible, thus enabling it to figure out what the most important factors are in getting students to pass.

Using knowledge about the data - for example, age, gender, family size, points (these are called *features*) we first plot them to establish relationships:

![Data Points](svm1.png)

For this drawing, the green dots are passed students, and the red ones are failed students.  The SVM algorithm then draws a boundary to try and seperate the passed students from the failed students:

![Data Points + Boundary](svm2.png)

We can see several plausable boundary lines here - but the objective of the SVM algorithm is to find the line with the maximum margin (solid line) and select that over a sub-optimal boundary (dashed lines).  By maximising the margin, we make the model as 'safe' as possible from error i.e. reduce passed students being mixed with failed and vice versa.

In reality, the data points are typically structured in much more complex ways, as with our student data.  We have to consider several features at once, and the relationships aren't so linear anymore:

![Complex Data + Boundary](svm3.png)

Here we can clearly see a curve or some other complex structure is required to seperate the data.  This means we have to do more than the basic SVM model and extend the functionality.  We achieve this by using mathematical functions, called 'kernels'.  What a kernel does is basically transform data points into a new dimension so that they can be seperable<sup>[7]</sup>:

![Kernel Trick 1](svm4.png)
![Kernel Trick 2](svm5.jpeg)

Here, the green points in particular are focused on and shifted onto a seperate dimension.  By then drawing a boundary across this dimension, the complex structure is now easily seperable by a linear boundary.

Once the boundary has been established, the model can use it to predict whether a student is going to pass or fail, depending on what side of the boundary a new data point is on.

<hr>

[7]M.  Elmezain, A.  Al-Hamadi, O.  Rashid and B.  Michaelis, "Posture and Gesture Recognition for Human-Computer Interaction", Advanced Technologies, 2009.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [44]:
# Import 'GridSearchCV' and 'make_scorer'
from sklearn.metrics import make_scorer
from sklearn.grid_search import GridSearchCV
from IPython.display import display
def f1_wrap(y_true, y_predict):
    return f1_score(y_true, y_predict, pos_label='yes')

# Create the parameters list you wish to tune.  Warning, takes ~15 seconds to compute!
parameters = {'C':range(1,6),'kernel':['linear','poly','rbf','sigmoid'],'degree':range(1,6)}

# Initialize the classifier
clf = SVC(random_state=0)

# Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_wrap)

# Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, scoring=f1_scorer)
# Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)
# Get the estimator
clf = grid_obj.best_estimator_

# Print the final parameters
df = pd.DataFrame(grid_obj.grid_scores_).sort_values('mean_validation_score').tail()
display(df)
print "Parameters for the optimal model: {}".format(clf.get_params())
# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train)[0])
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test)[0])

,parameters,mean_validation_score,cv_validation_scores
81,"{u'kernel': u'poly', u'C': 5, u'degree': 1}",0.806711,"[0.812903225806, 0.794326241135, 0.812903225806]"
61,"{u'kernel': u'poly', u'C': 4, u'degree': 1}",0.812546,"[0.815286624204, 0.794326241135, 0.828025477707]"
21,"{u'kernel': u'poly', u'C': 2, u'degree': 1}",0.813155,"[0.830188679245, 0.791666666667, 0.817610062893]"
41,"{u'kernel': u'poly', u'C': 3, u'degree': 1}",0.816004,"[0.822784810127, 0.797202797203, 0.828025477707]"
1,"{u'kernel': u'poly', u'C': 1, u'degree': 1}",0.816948,"[0.814814814815, 0.823529411765, 0.8125]"


Parameters for the optimal model: {'kernel': 'poly', 'C': 1, 'verbose': False, 'probability': False, 'degree': 1, 'shrinking': True, 'max_iter': -1, 'decision_function_shape': None, 'random_state': 0, 'tol': 0.001, 'cache_size': 200, 'coef0': 0.0, 'gamma': 'auto', 'class_weight': None}
Made predictions in 0.0100 seconds.
Tuned model has a training F1 score of 0.8305.
Made predictions in 0.0050 seconds.
Tuned model has a testing F1 score of 0.8105.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **  
- Final F<sub>1</sub> score for training: 0.8305
- Final F<sub>1</sub> score for testing: 0.8000
- Previous F<sub>1</sub> score for training: 0.9333
- Previous F<sub>1</sub> score for testing: 0.7832
- **Delta F<sub>1</sub> score for training: -0.1025**
- **Delta F<sub>1</sub> score for testing: +0.0168**

The training F<sub>1</sub> has actually *decreased* after optimazation.  This is unsurprising, and is due to the fact that the previous model would have been overfitted towards the training data.  This time, the training parameters have been cross-validated in order to make the training and testing scores more consistent.  Hence, although the training score has apparently decreased, it is actually less over-fitted and more consistent with testing.

The testing F<sub>1</sub> score has increased notably after optimization, now bringing the classification into the 80%s - a 1.7% gain.  This is a notable optimization, and certainly worth the minimal effort to tune the parameters.  However, it has not massively increased the accuracy which may be an indicator of the limitation of Support Vector Machines themselves - a different, more intensive model, might increase the accuracy further.

Overall, it forms a good predictive model for students in class and what factors contribute towards their pass and failure.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.